In [1]:
import pandas as pd
import numpy as np

from oudelaar_tiled_capture_2019 import config

In [2]:
exp_matrix = pd.read_csv(
    config.in_data_processed('cite-seq_mean_tpm_by_popn.csv'), index_col=0)

In [4]:
!grep -e "	gene	" ../../data/raw/annotation/genes.gtf | \
 awk '{ print $1"\t"$4"\t"$5"\t"$14"\t999\t"$7"\t"$10 }' | tr -d '";' > \
 ../../data/intermediate/hub/cite-seq/genes.bed

In [5]:
gene_locs = pd.read_csv('../../data/intermediate/hub/cite-seq/genes.bed',
            sep='\t', names=['chrom', 'start', 'stop', 'gene_name',
                             'score', 'strand', 'gene_id'])

In [6]:
num_genes = gene_locs.gene_name.value_counts()

unique_genes = num_genes[num_genes == 1].index.tolist()

In [7]:
scrna_genes = exp_matrix.index.tolist()

In [8]:
common_genes = set(scrna_genes).intersection(set(unique_genes))

In [10]:
random_chr_genes = gene_locs.loc[np.logical_not(
                        gene_locs.chrom.isin(
                            ['chr{}'.format(c) 
                             for c in list(range(1,20)) + ['X', 'Y', 'M']])),
                                 'gene_name'].tolist()

In [11]:
common_genes = list(common_genes - set(random_chr_genes))

In [13]:
exp_matrix.loc[common_genes, :].reset_index().to_csv(
    config.in_data_intermediate('hub/cite-seq/avg_counts.tsv'),
    sep='\t', index=False)

In [14]:
gene_locs[gene_locs.gene_name.isin(common_genes)].to_csv(
    config.in_data_intermediate('hub/cite-seq/gene_locs.bed'),
    sep='\t', index=False, header=False)

In [15]:
' '.join(exp_matrix.columns.tolist())

's0-low s0-med s1 s2 s3 s4/s5'

In [16]:
pd.DataFrame({'cluster': exp_matrix.columns.tolist(),
              'population': exp_matrix.columns.tolist()}).to_csv(
    config.in_data_intermediate('hub/cite-seq/sample_details.tsv'),
    sep='\t', index=False, header=False)

In [17]:
%%bash

module rm ucsctools
module load ucsctools/373

expMatrixToBarchartBed ../../data/intermediate/hub/cite-seq/sample_details.tsv \
 ../../data/intermediate/hub/cite-seq/avg_counts.tsv \
 ../../data/intermediate/hub/cite-seq/gene_locs.bed \
 ../../data/intermediate/hub/cite-seq/cite-seq_barchart.bed
 
sort -k1,1 -k2,2n ../../data/intermediate/hub/cite-seq/cite-seq_barchart.bed > \
 ../../data/intermediate/hub/cite-seq/cite-seq_barchart.sorted.bed
 

join: file 1 is not in sorted order
join: file 2 is not in sorted order


In [18]:
output_barchart = pd.read_csv(
    config.in_data_intermediate('hub/cite-seq/cite-seq_barchart.sorted.bed')
    , sep='\t')

In [26]:
!bedToBigBed -as=../../data/raw/barChartBed.as \
 -type=bed6+5 ../../data/intermediate/hub/cite-seq/cite-seq_barchart.sorted.bed \
 ../../data/raw/annotation/mm10.full.chrom.sizes \
 ../../data/processed/hub/bigbeds/citeseq_barchart.bb

pass1 - making usageList (22 chroms): 10 millis
pass2 - checking and writing primary data (14873 records, 11 fields): 105 millis
